<a href="https://colab.research.google.com/github/RemyaVKarthikeyan/AA-Stagecoach-Project/blob/main/18_07_2024_16_42_SWT%2C_SWT_hr1%2C_SWT_hr2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

18/07/2024

In [1]:
import pandas as pd
import requests
from difflib import SequenceMatcher

# Function to normalize stop names
def normalize_stop_name(name):
    return ' '.join(name.lower().split())

# Modify this path accordingly
file_path = '/content/QSI points.xlsx'

# Read the Excel file into a DataFrame
df = pd.read_excel(file_path)

# Get the lineID from the user
lineID = input("Please enter the lineID: ")

# Check if the necessary column is present
if 'Route_Dir_QSI_No' not in df.columns:
    print("The 'Route_Dir_QSI_No' column is not present in the provided file.")
else:
    # Convert the lineID to uppercase to ensure case-insensitivity
    lineID = lineID.upper()

    # Convert the 'Route_Dir_QSI_No' column to uppercase for comparison
    df['Route_Dir_QSI_No'] = df['Route_Dir_QSI_No'].str.upper()

    # Normalize the stop names in the DataFrame
    df['STOP_NAME'] = df['STOP_NAME'].apply(normalize_stop_name)

    # Create regular expressions for filtering
    pattern_A = f"^{lineID}_A\\d+$"  # Regular expression for lineID_A**
    pattern_B = f"^{lineID}_B\\d+$"  # Regular expression for lineID_B**

    # Filter rows where the 'Route_Dir_QSI_No' column matches the pattern
    filtered_df_A = df[df['Route_Dir_QSI_No'].str.match(pattern_A, na=False)][['Route_Dir_QSI_No', 'STOP_NAME']]
    filtered_df_B = df[df['Route_Dir_QSI_No'].str.match(pattern_B, na=False)][['Route_Dir_QSI_No', 'STOP_NAME']]

    # Function to fetch and process route sequence data from TfL API
    def fetch_and_process_route_data(route_type, pattern, filtered_df):
        api_url = f"https://api.tfl.gov.uk/Line/{lineID}/Route/Sequence/{route_type}"
        response = requests.get(api_url)

        results_list = []

        if response.status_code == 200:
            route_data = response.json()

            # Iterate through each stop in the route data
            for stop in route_data['stopPointSequences'][0]['stopPoint']:
                stop_name_api = normalize_stop_name(stop['name'])
                stop_id = stop['id']

                # Check if the stop_name_api exists in the filtered DataFrame for the correct direction
                matched_row = filtered_df[(filtered_df['STOP_NAME'] == stop_name_api) &
                                          (filtered_df['Route_Dir_QSI_No'].str.match(pattern))]

                if not matched_row.empty:
                    route_dir_qsi_no = matched_row.iloc[0]['Route_Dir_QSI_No']
                    results_list.append({
                        'Route_Dir_QSI_No': route_dir_qsi_no,
                        'STOP_Name': stop['name'],
                        'ID': stop_id
                    })
                else:
                    # If exact match not found, try partial matching based on words before and after '/'
                    api_stop_name_parts = stop_name_api.split('/')
                    for index, row in filtered_df.iterrows():
                        df_stop_name_parts = row['STOP_NAME'].split('/')
                        for api_part in api_stop_name_parts:
                            for df_part in df_stop_name_parts:
                                if SequenceMatcher(None, df_part.strip(), api_part.strip()).ratio() > 0.8:
                                    matched_row = pd.DataFrame([row])
                                    break
                            if not matched_row.empty:
                                break
                        if not matched_row.empty:
                            break

                    if not matched_row.empty:
                        route_dir_qsi_no = matched_row.iloc[0]['Route_Dir_QSI_No']
                        results_list.append({
                            'Route_Dir_QSI_No': route_dir_qsi_no,
                            'STOP_Name': stop['name'],
                            'ID': stop_id
                        })
        else:
            print(f"Failed to fetch route sequence data from TfL API for {route_type} route. Status code: {response.status_code}")

        return results_list

    # Fetch and process outbound route data for _A**
    matched_results_A = fetch_and_process_route_data('outbound', pattern_A, filtered_df_A)

    # Fetch and process inbound route data for _B**
    matched_results_B = fetch_and_process_route_data('inbound', pattern_B, filtered_df_B)

    # Create DataFrames from the matched results for each direction
    matched_results_df_A = pd.DataFrame(matched_results_A)
    matched_results_df_B = pd.DataFrame(matched_results_B)

    # Function to remove partial matches if exact matches are found
    def remove_partial_matches(exact_df, matched_df):
        for index, row in exact_df.iterrows():
            exact_stop_name = row['STOP_NAME']
            route_dir_qsi_no = row['Route_Dir_QSI_No']
            # Find exact matches in matched_df
            exact_matches = matched_df[(matched_df['Route_Dir_QSI_No'] == route_dir_qsi_no) &
                                       (matched_df['STOP_Name'].apply(normalize_stop_name) == exact_stop_name)]
            if not exact_matches.empty:
                # Remove partial matches
                matched_df = matched_df[~((matched_df['Route_Dir_QSI_No'] == route_dir_qsi_no) &
                                          (matched_df['STOP_Name'].apply(normalize_stop_name) != exact_stop_name))]
        return matched_df

    # Remove partial matches for direction A
    matched_results_df_A = remove_partial_matches(filtered_df_A, matched_results_df_A)

    # Remove partial matches for direction B
    matched_results_df_B = remove_partial_matches(filtered_df_B, matched_results_df_B)

    # Remove duplicate stop names with the same Route_Dir_QSI_No and different IDs
    matched_results_df_A = matched_results_df_A.drop_duplicates(subset=['Route_Dir_QSI_No', 'STOP_Name'], keep='first')
    matched_results_df_B = matched_results_df_B.drop_duplicates(subset=['Route_Dir_QSI_No', 'STOP_Name'], keep='first')

    # Print the matched results for direction A
    print(f"\n\n\033[1m\033[4mQSI stop points for direction {lineID}_A\033[0m\n")
    matched_results_df_A = matched_results_df_A[matched_results_df_A['Route_Dir_QSI_No'].str.match(pattern_A)]
    print(matched_results_df_A[['Route_Dir_QSI_No', 'STOP_Name', 'ID']])

    # Print the matched results for direction B
    print(f"\n\n\033[1m\033[4mQSI stop points for direction {lineID}_B\033[0m\n")
    matched_results_df_B = matched_results_df_B[matched_results_df_B['Route_Dir_QSI_No'].str.match(pattern_B)]
    print(matched_results_df_B[['Route_Dir_QSI_No', 'STOP_Name', 'ID']])

    # Concatenate the matched results DataFrames for directions A and B
    combined_df = pd.concat([matched_results_df_A, matched_results_df_B], ignore_index=True)

    # Display the combined DataFrame
    print("\n\n\033[1m\033[4mCombined QSI stop points for directions A and B\033[0m\n")
    print(combined_df[['Route_Dir_QSI_No', 'STOP_Name', 'ID']])


Please enter the lineID: 179


QSI stop points for direction 179_A

  Route_Dir_QSI_No                            STOP_Name          ID
0           179_A1                    Chingford Station  490001063C
2           179_A2                          Friday Hill  490007021E
3           179_A3      Woodford Green / Broadmead Road  490014847S
4           179_A4               South Woodford Station  490015122C
5           179_A5           Charlie Brown's Roundabout  490005002N
7           179_A6  Gants Hill Station / Cranbrook Road  490015202N
8           179_A7                       Ilford Station  490005699D


QSI stop points for direction 179_B

  Route_Dir_QSI_No                            STOP_Name          ID
0           179_B1                      Hainault Street  490007657W
1           179_B2                       Ilford Station  490001157G
2           179_B3  Gants Hill Station / Cranbrook Road  490015202M
4           179_B4                        Southend Road  490012285S
5        

In [2]:
from datetime import datetime, timedelta
import pytz
import time  # Import the time module for sleep functionality

# Function to fetch data from the API
def fetch_data(url):
    response = requests.get(url)
    return response.json()

# Function to extract schedule names
def extract_schedule_names(data, schedule_names_dict={}):
    if isinstance(data, dict):
        if data.get('$type') == "Tfl.Api.Presentation.Entities.Schedule, Tfl.Api.Presentation.Entities" and 'knownJourneys' in data:
            if 'name' in data:
                schedule_names_dict[data['name']] = data['knownJourneys']
        for key, value in data.items():
            extract_schedule_names(value, schedule_names_dict)
    elif isinstance(data, list):
        for item in data:
            extract_schedule_names(item, schedule_names_dict)
    return schedule_names_dict

# Function to categorize journeys into hourly slots
def categorize_into_slots(timetable):
    slots = [[] for _ in range(24)]
    for journey in timetable:
        hour = int(journey['hour'])  # Convert hour to integer
        if 0 <= hour < 24:  # Ensure hour is within the valid range
            slots[hour].append(journey)
    return slots

# Function to fetch the current day of the week
def get_day_of_week():
    bst = pytz.timezone('Europe/London')
    now = datetime.now(bst)
    return now.strftime('%A')  # %A gives full weekday name (e.g., 'Monday')

# DataFrames to store SWT data
swt_data = {
    'Route_Dir_QSI_No': [],
    'ID': [],
    'SWT': [],
    'No_Buses_Sch': [],
    'SWT_hr1': [],
    'No_Buses_Sch_hr1': [],
    'SWT_hr2': [],
    'No_Buses_Sch_hr2': []
}

# Fetch timetable for each stop point ID and calculate SWT
bst = pytz.timezone('Europe/London')
while True:
    # Update current time and hour
    current_time = datetime.now(bst)
    current_hour = current_time.hour
    day_of_week = get_day_of_week()

    # Store selected schedule name to ensure it's printed only once
    selected_schedule_name = None
    printed_schedule_name = False

    for index, row in combined_df.iterrows():
        stop_point_id = row['ID']
        route_dir_qsi_no = row['Route_Dir_QSI_No']

        if f"{lineID}_A" in route_dir_qsi_no:
            direction = 'outbound'
        elif f"{lineID}_B" in route_dir_qsi_no:
            direction = 'inbound'
        else:
            continue

        url = f'https://api.tfl.gov.uk/Line/{lineID}/Timetable/{stop_point_id}?direction={direction}'
        data = fetch_data(url)

        schedule_names_dict = extract_schedule_names(data)

        if not selected_schedule_name:
            if day_of_week.lower() in ['monday', 'tuesday', 'wednesday', 'thursday']:
                preferred_schedule_names = ['Mon-Fri Schooldays', 'Monday to Thursday', 'Monday to Friday']
            elif day_of_week.lower() == 'friday':
                preferred_schedule_names = ['Mon-Fri Schooldays', 'Monday to Friday', 'Friday']
            elif day_of_week.lower() == 'saturday':
                preferred_schedule_names = ['Saturday']
            elif day_of_week.lower() == 'sunday':
                preferred_schedule_names = ['Sunday']
            else:
                preferred_schedule_names = [day_of_week]

            for preferred_name in preferred_schedule_names:
                if preferred_name in schedule_names_dict:
                    selected_schedule_name = preferred_name
                    break

        if selected_schedule_name and not printed_schedule_name:
            print(f"\n\033[1m\033[4mToday is {day_of_week}. The selected Schedule name is {selected_schedule_name}.\033[0m")
            printed_schedule_name = True

        if selected_schedule_name:
            timetable = schedule_names_dict[selected_schedule_name]
            slots = categorize_into_slots(timetable)

            # Calculate Scheduled Wait Time (SWT)
            total_buses_this_hour = len(slots[current_hour])
            scheduled_wait_time = 60 / (total_buses_this_hour * 2) if total_buses_this_hour > 0 else float('inf')
            next_hour = (current_hour + 1) % 24  # Wrap around using modulo operator
            total_buses_hr1 = len(slots[next_hour])
            scheduled_wait_time_hr1 = 60 / (total_buses_hr1 * 2) if total_buses_hr1 > 0 else float('inf')
            next_next_hour = (current_hour + 2) % 24  # Wrap around using modulo operator
            total_buses_hr2 = len(slots[next_next_hour])
            scheduled_wait_time_hr2 = 60 / (total_buses_hr2 * 2) if total_buses_hr2 > 0 else float('inf')

            # Store SWT data
            swt_data['Route_Dir_QSI_No'].append(route_dir_qsi_no)
            swt_data['ID'].append(stop_point_id)
            swt_data['SWT'].append(scheduled_wait_time)
            swt_data['No_Buses_Sch'].append(total_buses_this_hour)
            swt_data['SWT_hr1'].append(scheduled_wait_time_hr1)
            swt_data['No_Buses_Sch_hr1'].append(total_buses_hr1)
            swt_data['SWT_hr2'].append(scheduled_wait_time_hr2)
            swt_data['No_Buses_Sch_hr2'].append(total_buses_hr2)

    # Create DataFrame for SWT data
    swt_df = pd.DataFrame(swt_data)

    # Print the SWT DataFrame
    #print ("\n_____________________________________________")
    print("\n\nSWT DataFrame:")
    print(swt_df)
    #print ("_____________________________________________\n")
    # Wait for the next hour (3600 seconds = 1 hour)
    time.sleep(3600)



Today is Thursday. The selected Schedule name is Mon-Fri Schooldays.


SWT DataFrame:
   Route_Dir_QSI_No          ID   SWT  No_Buses_Sch  SWT_hr1  \
0            179_A1  490001063C  10.0             3     10.0   
1            179_A2  490007021E  10.0             3     10.0   
2            179_A3  490014847S  10.0             3     10.0   
3            179_A4  490015122C  10.0             3     10.0   
4            179_A5  490005002N  10.0             3     10.0   
5            179_A6  490015202N  10.0             3     10.0   
6            179_A7  490005699D   7.5             4     10.0   
7            179_B1  490007657W  10.0             3     10.0   
8            179_B2  490001157G  10.0             3     10.0   
9            179_B3  490015202M  10.0             3     10.0   
10           179_B4  490012285S  10.0             3     10.0   
11           179_B5  490000217E  10.0             3     10.0   
12           179_B6  490014847N  10.0             3     10.0   
13           179_

KeyboardInterrupt: 

In [3]:
import pandas as pd
import requests
from datetime import datetime
import pytz
import time

# Function to normalize stop names
def normalize_stop_name(name):
    return ' '.join(name.lower().split())

# Function to fetch data from the API
def fetch_data(url):
    response = requests.get(url)
    return response.json()

# Function to extract schedule names
def extract_schedule_names(data, schedule_names_dict={}):
    if isinstance(data, dict):
        if data.get('$type') == "Tfl.Api.Presentation.Entities.Schedule, Tfl.Api.Presentation.Entities" and 'knownJourneys' in data:
            if 'name' in data:
                schedule_names_dict[data['name']] = data['knownJourneys']
        for key, value in data.items():
            extract_schedule_names(value, schedule_names_dict)
    elif isinstance(data, list):
        for item in data:
            extract_schedule_names(item, schedule_names_dict)
    return schedule_names_dict

# Function to categorize journeys into hourly slots
def categorize_into_slots(timetable):
    slots = [[] for _ in range(24)]
    for journey in timetable:
        hour = int(journey['hour'])  # Convert hour to integer
        if 0 <= hour < 24:  # Ensure hour is within the valid range
            slots[hour].append(journey)
    return slots

# Function to fetch the current day of the week
def get_day_of_week():
    bst = pytz.timezone('Europe/London')
    now = datetime.now(bst)
    return now.strftime('%A')  # %A gives full weekday name (e.g., 'Monday')

# Function to calculate Scheduled Wait Time (SWT) for all hours
def calculate_swt_for_all_hours(slots):
    swt_per_hour = []
    for hour in range(24):
        total_buses = len(slots[hour])
        scheduled_wait_time = 60 / (total_buses * 2) if total_buses > 0 else float('inf')
        swt_per_hour.append((scheduled_wait_time, total_buses))
    return swt_per_hour

# Function to select the preferred schedule name based on the current day of the week
def select_preferred_schedule(schedule_names_dict, day_of_week):
    if day_of_week.lower() in ['monday', 'tuesday', 'wednesday', 'thursday']:
        preferred_schedule_names = ['Mon-Fri Schooldays', 'Monday to Thursday', 'Monday to Friday']
    elif day_of_week.lower() == 'friday':
        preferred_schedule_names = ['Mon-Fri Schooldays', 'Monday to Friday', 'Friday']
    elif day_of_week.lower() == 'saturday':
        preferred_schedule_names = ['Saturday']
    elif day_of_week.lower() == 'sunday':
        preferred_schedule_names = ['Sunday']
    else:
        preferred_schedule_names = [day_of_week]

    for preferred_name in preferred_schedule_names:
        if preferred_name in schedule_names_dict:
            return preferred_name
    return None

# Main logic to fetch and calculate SWT
def main(combined_df, lineID):
    bst = pytz.timezone('Europe/London')
    swt_data = {
        'Route_Dir_QSI_No': [],
        'ID': [],
    }
    # Initialize keys for all 24 hours in the dictionary
    for hour in range(24):
        swt_data[f'SWT_hr{hour}'] = []
        swt_data[f'No_Buses_Sch_hr{hour}'] = []

    while True:
        # Update current time and hour
        current_time = datetime.now(bst)
        day_of_week = get_day_of_week()

        # Store selected schedule name to ensure it's printed only once
        selected_schedule_name = None
        printed_schedule_name = False

        for index, row in combined_df.iterrows():
            stop_point_id = row['ID']
            route_dir_qsi_no = row['Route_Dir_QSI_No']

            if f"{lineID}_A" in route_dir_qsi_no:
                direction = 'outbound'
            elif f"{lineID}_B" in route_dir_qsi_no:
                direction = 'inbound'
            else:
                continue

            url = f'https://api.tfl.gov.uk/Line/{lineID}/Timetable/{stop_point_id}?direction={direction}'
            data = fetch_data(url)

            schedule_names_dict = extract_schedule_names(data)

            if not selected_schedule_name:
                selected_schedule_name = select_preferred_schedule(schedule_names_dict, day_of_week)

            if selected_schedule_name and not printed_schedule_name:
                print(f"\n\033[1m\033[4mToday is {day_of_week}. The selected Schedule name is {selected_schedule_name}.\033[0m")
                printed_schedule_name = True

            if selected_schedule_name:
                timetable = schedule_names_dict[selected_schedule_name]
                slots = categorize_into_slots(timetable)

                # Calculate SWT for all hours
                swt_per_hour = calculate_swt_for_all_hours(slots)

                # Store SWT data for all hours
                swt_data['Route_Dir_QSI_No'].append(route_dir_qsi_no)
                swt_data['ID'].append(stop_point_id)
                for hour in range(24):
                    swt, total_buses = swt_per_hour[hour]
                    swt_data[f'SWT_hr{hour}'].append(swt)
                    swt_data[f'No_Buses_Sch_hr{hour}'].append(total_buses)

        # Create DataFrame for SWT data
        swt_df = pd.DataFrame(swt_data)

        # Print the SWT DataFrame
        print("\n\nSWT DataFrame:")
        print(swt_df)

        # Wait for the next hour (3600 seconds = 1 hour)
        time.sleep(3600)

# Assuming combined_df and lineID are defined and available
# combined_df = ... (from previous code)
# lineID = ... (from user input)
main(combined_df, lineID)



Today is Thursday. The selected Schedule name is Mon-Fri Schooldays.


SWT DataFrame:
   Route_Dir_QSI_No          ID  SWT_hr0  No_Buses_Sch_hr0  SWT_hr1  \
0            179_A1  490001063C      inf                 0      inf   
1            179_A2  490007021E      inf                 0      inf   
2            179_A3  490014847S      inf                 0      inf   
3            179_A4  490015122C      inf                 0      inf   
4            179_A5  490005002N      inf                 0      inf   
5            179_A6  490015202N      inf                 0      inf   
6            179_A7  490005699D      inf                 0      inf   
7            179_B1  490007657W      inf                 0      inf   
8            179_B2  490001157G      inf                 0      inf   
9            179_B3  490015202M      inf                 0      inf   
10           179_B4  490012285S      inf                 0      inf   
11           179_B5  490000217E      inf                 0   

KeyboardInterrupt: 

In [4]:
import pandas as pd
import requests
from datetime import datetime
import pytz
import time

# Function to normalize stop names
def normalize_stop_name(name):
    return ' '.join(name.lower().split())

# Function to fetch data from the API
def fetch_data(url):
    response = requests.get(url)
    return response.json()

# Function to extract schedule names
def extract_schedule_names(data, schedule_names_dict={}):
    if isinstance(data, dict):
        if data.get('$type') == "Tfl.Api.Presentation.Entities.Schedule, Tfl.Api.Presentation.Entities" and 'knownJourneys' in data:
            if 'name' in data:
                schedule_names_dict[data['name']] = data['knownJourneys']
        for key, value in data.items():
            extract_schedule_names(value, schedule_names_dict)
    elif isinstance(data, list):
        for item in data:
            extract_schedule_names(item, schedule_names_dict)
    return schedule_names_dict

# Function to categorize journeys into hourly slots
def categorize_into_slots(timetable):
    slots = [[] for _ in range(24)]
    for journey in timetable:
        hour = int(journey['hour'])  # Convert hour to integer
        if 0 <= hour < 24:  # Ensure hour is within the valid range
            slots[hour].append(journey)
    return slots

# Function to fetch the current day of the week
def get_day_of_week():
    bst = pytz.timezone('Europe/London')
    now = datetime.now(bst)
    return now.strftime('%A')  # %A gives full weekday name (e.g., 'Monday')

# Function to calculate Scheduled Wait Time (SWT) for all hours
def calculate_swt_for_all_hours(slots):
    swt_per_hour = []
    for hour in range(24):
        total_buses = len(slots[hour])
        scheduled_wait_time = 60 / (total_buses * 2) if total_buses > 0 else None  # Use None to indicate no buses
        swt_per_hour.append((scheduled_wait_time, total_buses))
    return swt_per_hour

# Function to select the preferred schedule name based on the current day of the week
def select_preferred_schedule(schedule_names_dict, day_of_week):
    if day_of_week.lower() in ['monday', 'tuesday', 'wednesday', 'thursday']:
        preferred_schedule_names = ['Mon-Fri Schooldays', 'Monday to Thursday', 'Monday to Friday']
    elif day_of_week.lower() == 'friday':
        preferred_schedule_names = ['Mon-Fri Schooldays', 'Monday to Friday', 'Friday']
    elif day_of_week.lower() == 'saturday':
        preferred_schedule_names = ['Saturday']
    elif day_of_week.lower() == 'sunday':
        preferred_schedule_names = ['Sunday']
    else:
        preferred_schedule_names = [day_of_week]

    for preferred_name in preferred_schedule_names:
        if preferred_name in schedule_names_dict:
            return preferred_name
    return None

# Main logic to fetch and calculate SWT
def main(combined_df, lineID):
    bst = pytz.timezone('Europe/London')
    swt_data = {
        'Route_Dir_QSI_No': [],
        'ID': [],
    }
    # Initialize keys for all 24 hours in the dictionary
    for hour in range(24):
        swt_data[f'SWT_{hour}'] = []
        swt_data[f'Sch_{hour}'] = []

    while True:
        # Update current time and hour
        current_time = datetime.now(bst)
        day_of_week = get_day_of_week()

        # Store selected schedule name to ensure it's printed only once
        selected_schedule_name = None
        printed_schedule_name = False

        for index, row in combined_df.iterrows():
            stop_point_id = row['ID']
            route_dir_qsi_no = row['Route_Dir_QSI_No']

            if f"{lineID}_A" in route_dir_qsi_no:
                direction = 'outbound'
            elif f"{lineID}_B" in route_dir_qsi_no:
                direction = 'inbound'
            else:
                continue

            url = f'https://api.tfl.gov.uk/Line/{lineID}/Timetable/{stop_point_id}?direction={direction}'
            data = fetch_data(url)

            schedule_names_dict = extract_schedule_names(data)

            if not selected_schedule_name:
                selected_schedule_name = select_preferred_schedule(schedule_names_dict, day_of_week)

            if selected_schedule_name and not printed_schedule_name:
                print(f"\n\033[1m\033[4mToday is {day_of_week}. The selected Schedule name is {selected_schedule_name}.\033[0m")
                printed_schedule_name = True

            if selected_schedule_name:
                timetable = schedule_names_dict[selected_schedule_name]
                slots = categorize_into_slots(timetable)

                # Calculate SWT for all hours
                swt_per_hour = calculate_swt_for_all_hours(slots)

                # Store SWT data for all hours
                swt_data['Route_Dir_QSI_No'].append(route_dir_qsi_no)
                swt_data['ID'].append(stop_point_id)
                for hour in range(24):
                    swt, total_buses = swt_per_hour[hour]
                    swt_data[f'SWT_{hour}'].append(swt)
                    swt_data[f'Sch_{hour}'].append(total_buses)

        # Create DataFrame for SWT data
        swt_df = pd.DataFrame(swt_data)

        # Print the SWT DataFrame
        print("\n\nSWT DataFrame:")
        print(swt_df)

        # Wait for the next hour (3600 seconds = 1 hour)
        time.sleep(3600)

# Assuming combined_df and lineID are defined and available
# combined_df = ... (from previous code)
# lineID = ... (from user input)
main(combined_df, lineID)



Today is Thursday. The selected Schedule name is Mon-Fri Schooldays.


SWT DataFrame:
   Route_Dir_QSI_No          ID SWT_0  Sch_0 SWT_1  Sch_1 SWT_2  Sch_2 SWT_3  \
0            179_A1  490001063C  None      0  None      0  None      0  None   
1            179_A2  490007021E  None      0  None      0  None      0  None   
2            179_A3  490014847S  None      0  None      0  None      0  None   
3            179_A4  490015122C  None      0  None      0  None      0  None   
4            179_A5  490005002N  None      0  None      0  None      0  None   
5            179_A6  490015202N  None      0  None      0  None      0  None   
6            179_A7  490005699D  None      0  None      0  None      0  None   
7            179_B1  490007657W  None      0  None      0  None      0  None   
8            179_B2  490001157G  None      0  None      0  None      0  None   
9            179_B3  490015202M  None      0  None      0  None      0  None   
10           179_B4  490012285S  

KeyboardInterrupt: 